In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
img_folder='/content/drive/MyDrive/sprints project /Scenes training set'
import splitfolders
splitfolders.ratio(img_folder, output="output",
    seed=1337, ratio=(.7,0.3), group_prefix=None, move=False)

Copying files: 14034 files [04:50, 48.31 files/s] 


In [ ]:
train_dir = '/content/output/train'
val_dir = '/content/output/val'
test_dir = '/content/drive/MyDrive/sprints project /testset'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
 
val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_dir,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
 
val_set = val_datagen.flow_from_directory(val_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            shuffle=False,
                                            class_mode = 'categorical')

Found 9820 images belonging to 6 classes.
Found 4214 images belonging to 6 classes.
Found 3050 images belonging to 1 classes.


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

import numpy as np

np.random.seed(1000)

AlexNet = Sequential()
AlexNet.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(224,224,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(6))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 56, 56, 96)        34944     
                                                                 
 batch_normalization_30 (Bat  (None, 56, 56, 96)       384       
 chNormalization)                                                
                                                                 
 activation_26 (Activation)  (None, 56, 56, 96)        0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 28, 28, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 28, 28, 256)       614656    
                                                                 
 batch_normalization_31 (Bat  (None, 28, 28, 256)     

In [ ]:
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
AlexNet.fit(training_set,
          batch_size=128,
          epochs=25,
          verbose=1,
          validation_data=val_set)

Epoch 1/25
307/307 [==============================] - 191s 401ms/step - loss: 1.0447 - accuracy: 0.6213 - val_loss: 1.5279 - val_accuracy: 0.4535
Epoch 2/25
307/307 [==============================] - 120s 392ms/step - loss: 0.8376 - accuracy: 0.7137 - val_loss: 2.0129 - val_accuracy: 0.3640
Epoch 3/25
307/307 [==============================] - 119s 387ms/step - loss: 0.7286 - accuracy: 0.7557 - val_loss: 1.9729 - val_accuracy: 0.3446
Epoch 4/25
307/307 [==============================] - 121s 395ms/step - loss: 0.6757 - accuracy: 0.7740 - val_loss: 1.4781 - val_accuracy: 0.4922
Epoch 5/25
307/307 [==============================] - 119s 388ms/step - loss: 0.6164 - accuracy: 0.7941 - val_loss: 1.0605 - val_accuracy: 0.6125
Epoch 6/25
307/307 [==============================] - 120s 390ms/step - loss: 0.5981 - accuracy: 0.7988 - val_loss: 1.1634 - val_accuracy: 0.5942
Epoch 7/25
307/307 [==============================] - 118s 385ms/step - loss: 0.5493 - accuracy: 0.8148 - val_loss: 0.7395 -

In [ ]:
AlexNet.fit(training_set,
          batch_size=128,
          epochs=3,
          verbose=1,
          validation_data=val_set)

Epoch 1/3
307/307 [==============================] - 123s 400ms/step - loss: 0.2605 - accuracy: 0.9123 - val_loss: 0.6824 - val_accuracy: 0.7667
Epoch 2/3
307/307 [==============================] - 121s 395ms/step - loss: 0.2406 - accuracy: 0.9175 - val_loss: 0.4733 - val_accuracy: 0.8403
Epoch 3/3
307/307 [==============================] - 116s 378ms/step - loss: 0.2360 - accuracy: 0.9196 - val_loss: 0.7735 - val_accuracy: 0.7430


In [ ]:
STEP_SIZE_VALID=val_set.n//val_set.batch_size
AlexNet.evaluate_generator(generator=val_set,
steps= STEP_SIZE_VALID)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


[0.77260422706604, 0.7437977194786072]

In [ ]:
STEP_SIZE_TRAIN=training_set.n//training_set.batch_size
AlexNet.evaluate_generator(generator=training_set,
steps=STEP_SIZE_TRAIN)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


[0.5438219308853149, 0.805657684803009]

In [ ]:
import os 
import glob
import pandas as pd

Name = []

for img in glob.glob(test_dir):
    prediction = AlexNet.predict_generator(test_set,steps=130,verbose=1)
    for name in os.listdir('/content/drive/MyDrive/sprints project /testset/Scenes testing test'):
         Name.append(name[0::])



predicted_class_indics=np.argmax(prediction,axis=1)
Name.sort()
Temp = {'Image':Name, 'Label':predicted_class_indics}
temp = pd.DataFrame(Temp)
temp.to_csv('/content/results.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


 95/130 [====================>.........] - ETA: 3s

130/130 [==============================] - 9s 71ms/step
